
## Projeto ICMC Júnior
### Objetivo: Prever se um funcionário vai sair da empresa (attrition).

In [13]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [14]:
# Leitura do arquivo csv
df = pd.read_csv('dados.csv')
# Verificação do tamanho da base de dados
df.shape

(1470, 35)

In [15]:
# Exibe os 5 primeiros funcionários da base de dados, apenas para contextualizar com a base
df.head()

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,1102,Sales,1.0,2,Life Sciences,1.0,1,...,1,80,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,279,Research & Development,8.0,1,Life Sciences,1.0,2,...,4,80,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,1373,Research & Development,2.0,2,Other,1.0,4,...,2,80,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,1392,Research & Development,NaN,4,Life Sciences,NaN,5,...,3,80,0,8,3,3,8,7,3,0
4,27,No,NaN,591,Research & Development,2.0,1,Medical,NaN,7,...,4,80,1,6,3,3,2,2,2,2


In [16]:
# TRATAMENTO DE VALORES NULOS
df.isnull().sum() # Verifica quantos valores nulos tem por colunas, ou seja, por categoria.

Age                           0
Attrition                     0
BusinessTravel               70
DailyRate                     0
Department                    0
DistanceFromHome             47
Education                     0
EducationField                0
EmployeeCount               220
EmployeeNumber                0
EnvironmentSatisfaction      53
Gender                       67
HourlyRate                   32
JobInvolvement               61
JobLevel                      0
JobRole                       0
JobSatisfaction               0
MaritalStatus                 0
MonthlyIncome                 0
MonthlyRate                   0
NumCompaniesWorked            0
Over18                        0
OverTime                     51
PercentSalaryHike             0
PerformanceRating            46
RelationshipSatisfaction      0
StandardHours                 0
StockOptionLevel              0
TotalWorkingYears             0
TrainingTimesLastYear         0
WorkLifeBalance               0
YearsAtC

In [17]:
df['EmployeeCount']

0       1.0
1       1.0
2       1.0
3       NaN
4       NaN
       ... 
1465    1.0
1466    1.0
1467    1.0
1468    1.0
1469    1.0
Name: EmployeeCount, Length: 1470, dtype: float64

In [18]:
# Em 'EmployeeCount' todos os valores são como padrão igual a 1.0, sendo cada linha, um funcionário, com isso se corrige os NaN para 1.0
# no dataframe original
df['EmployeeCount'].fillna(value=1, inplace=True)

In [19]:
# Tratamento dos valores nulos da coluna: DistanceFromHome
# Calculando o IQR = Q3 - Q1
Q1 = df['DistanceFromHome'].quantile(0.25) # 1° Quartil
Q3 = df['DistanceFromHome'].quantile(0.75) # 3° Quartil


IQR = Q3 - Q1 # Amplitude interquartil
limite_superior = Q3 + 1.5 * IQR
limite_inferior = Q1 - 1.5 * IQR

print(f"Q1: {Q1}")
print(f"Q3: {Q3}")
print(f"IQR (Q3 - Q1): {IQR}")
print(f"----------------------------------")
print(f"Limite Inferior (Q1 - 1.5*IQR): {limite_inferior}")
print(f"Limite Superior (Q3 + 1.5*IQR): {limite_superior}")

Q1: 2.0
Q3: 14.0
IQR (Q3 - Q1): 12.0
----------------------------------
Limite Inferior (Q1 - 1.5*IQR): -16.0
Limite Superior (Q3 + 1.5*IQR): 32.0


In [ ]:
# Resumo das descreições estatísticas da categoria DistanceFromHome
df['DistanceFromHome'].describe()

count    1423.000000
mean        9.188335
std         8.133707
min         1.000000
25%         2.000000
50%         7.000000
75%        14.000000
max        29.000000
Name: DistanceFromHome, dtype: float64

In [21]:
# O menor valor (min) é igual a 1.0 e o maior valor (max) é igual a 29.0, com isso podemos ver atráves disso, todos 
# os dados então dentro das margens, não tendo nenhum outliers. No entanto, devido ao desvio padrão ser mais de 80% da média, 
# indica que meus dados, não estão bem distribuidos. Utilizar a mediana é a melhor representação dos dados.

# Calculo a mediana
median_distances = df['DistanceFromHome'].median() 
# Preenche todos os valores NaN com a mediana e em seguida, reatribui a coluna nela mesma.
df['DistanceFromHome'] = df['DistanceFromHome'].fillna(median_distances)


In [22]:
df['DistanceFromHome']

0       1.0
1       8.0
2       2.0
3       7.0
4       2.0
       ... 
1465    7.0
1466    6.0
1467    4.0
1468    2.0
1469    8.0
Name: DistanceFromHome, Length: 1470, dtype: float64